<a href="https://colab.research.google.com/github/ichi-a00/google_colab/blob/main/ikawidget2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

スプラトゥーン２の戦績データから主成分分析を行います。
参照：https://ossyaritoori.hatenablog.com/entry/2020/04/30/%E2%91%A0_ikawidget2%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6Splatoon2%E3%81%A7%E3%81%AE%E7%B7%A8%E6%88%90%E3%82%92%E8%A9%95%E4%BE%A1_%EF%BD%9E%EF%BC%91%EF%BC%8E

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [11]:
csv = pd.read_csv("/content/ikaWidgetCSV_20211227140557_.csv")

In [27]:
csv.keys()

Index(['no', 'startTime', 'elapsedTime', 'win', 'udemae', 'udemaeName',
       'udemaeIsX', 'udemaeIsReached', 'sPlusNumber', 'xRanking',
       ...
       'bravo4ShoesSkillName', 'bravo4ShoesSub1SkillID',
       'bravo4ShoesSub1SkillName', 'bravo4ShoesSub2SkillID',
       'bravo4ShoesSub2SkillName', 'bravo4ShoesSub3SkillID',
       'bravo4ShoesSub3SkillName', 'Unnamed: 637', 'Unnamed: 638',
       'Unnamed: 639'],
      dtype='object', length=640)

In [23]:
csv['alpha1AllKill']

0      10
1      21
2      10
3      14
4       8
       ..
427    11
428     1
429    15
430     8
431    15
Name: alpha1AllKill, Length: 432, dtype: int64